In [10]:
import duckdb
import pandas as pd
import json
import argparse
import sys
import os
import time

def load_geo_lookup(parquet_path):
    """
    Reads the Parquet file using DuckDB, extracts 4-digit postcodes
    and their corresponding lat/lon coordinates, and returns them as a dictionary.

    Args:
        parquet_path (str): Path to the GeoParquet file.

    Returns:
        dict: A dictionary mapping pc4_code (str) to {'latitude': float, 'longitude': float}.
              Returns None if an error occurs.
    """
    print(f"Loading geospatial data from: {parquet_path}...")
    if not os.path.exists(parquet_path):
        print(f"Error: Parquet file not found at '{parquet_path}'", file=sys.stderr)
        return None

    con = None
    geo_lookup = {}
    try:
        con = duckdb.connect(database=':memory:', 
                             config={'allow_unsigned_extensions': 'true'})

        # Load spatial extension (required for ST_X, ST_Y)
        try:
            con.execute("INSTALL spatial;")
            con.execute("LOAD spatial;")
            print("Spatial extension loaded.")
            spatial_present = True
        except Exception as e:
            print(f"Warning: Could not load spatial extension ({e}). Cannot extract lat/lon.", file=sys.stderr)
            return None # Cannot proceed without spatial functions

        # Check if necessary columns exist (basic check)
        schema_df = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{parquet_path}') LIMIT 1;").df()
        required_cols = {'pc4_code', 'geo_point_2d'}
        if not required_cols.issubset(set(schema_df['column_name'])):
             print(f"Error: Parquet file must contain columns 'pc4_code' and 'geo_point_2d'. Found: {schema_df['column_name'].tolist()}", file=sys.stderr)
             return None

        # Query to get postcode and coordinates
        query = f"""
        SELECT
            pc4_code,
            ST_Y(geo_point_2d) AS latitude,
            ST_X(geo_point_2d) AS longitude
        FROM read_parquet('{parquet_path}')
        WHERE pc4_code IS NOT NULL AND geo_point_2d IS NOT NULL;
        """

        print("Executing DuckDB query to extract pc4 and coordinates...")
        start_time = time.time()
        geo_data_df = con.execute(query).df()
        end_time = time.time()
        print(f"Query finished in {end_time - start_time:.2f} seconds. Processing {len(geo_data_df)} rows.")

        # Create the lookup dictionary
        # Ensure pc4_code is string, handle potential duplicates by keeping the first
        geo_data_df = geo_data_df.dropna(subset=['pc4_code', 'latitude', 'longitude'])
        geo_data_df['pc4_code'] = geo_data_df['pc4_code'].astype(str)
        # If duplicates matter, decide strategy (e.g., log warning, average). Keep first is simple.
        geo_data_df = geo_data_df.drop_duplicates(subset=['pc4_code'], keep='first')

        # Convert DataFrame to the desired dictionary format
        geo_lookup = geo_data_df.set_index('pc4_code')[['latitude', 'longitude']].to_dict('index')

        print(f"Created lookup dictionary with {len(geo_lookup)} unique 4-digit postcodes.")
        return geo_lookup

    except Exception as e:
        print(f"\nAn error occurred while loading geo data: {e}", file=sys.stderr)
        print(f"Error Type: {type(e)}", file=sys.stderr)
        return None
    finally:
        if con:
            con.close()
            print("DuckDB connection closed.")


def extract_pc4(postcode_full):
    """
    Safely extracts the first 4 digits from a Dutch postcode string.

    Args:
        postcode_full (str or None): The full postcode string (e.g., "6663 CP").

    Returns:
        str or None: The 4-digit postcode (e.g., "6663") or None if invalid/missing.
    """
    if not postcode_full or not isinstance(postcode_full, str):
        return None
    # Remove leading/trailing whitespace
    cleaned_postcode = postcode_full.strip()
    if len(cleaned_postcode) >= 4 and cleaned_postcode[:4].isdigit():
        return cleaned_postcode[:4]
    else:
        # Optionally, try regex for more flexible matching:
        # import re
        # match = re.match(r"^\s*(\d{4})", cleaned_postcode)
        # if match:
        #     return match.group(1)
        return None # Return None if first 4 chars are not digits or length < 4


def add_geo_to_json(json_input_path, geo_lookup, json_output_path):
    """
    Reads the input JSON, adds lat/lon from the geo_lookup, and writes to output JSON.

    Args:
        json_input_path (str): Path to the input JSON file.
        geo_lookup (dict): Dictionary mapping pc4_code to {'latitude': y, 'longitude': x}.
        json_output_path (str): Path to save the augmented JSON file.
    """
    print(f"\nReading input JSON: {json_input_path}...")
    try:
        with open(json_input_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
        print(f"Read {len(json_data)} records from JSON.")
    except FileNotFoundError:
        print(f"Error: Input JSON file not found at '{json_input_path}'", file=sys.stderr)
        return
    except json.JSONDecodeError as e:
        print(f"Error: Could not decode input JSON file '{json_input_path}': {e}", file=sys.stderr)
        return
    except Exception as e:
        print(f"Error reading input JSON file '{json_input_path}': {e}", file=sys.stderr)
        return

    print("Augmenting JSON records with coordinates...")
    match_count = 0
    no_postcode_count = 0
    no_match_count = 0

    for i, record in enumerate(json_data):
        full_postcode = record.get('Postcode')
        pc4 = extract_pc4(full_postcode)

        if pc4:
            coords = geo_lookup.get(pc4) # Look up using the 4-digit string
            if coords:
                record['latitude'] = coords['latitude']
                record['longitude'] = coords['longitude']
                match_count += 1
            else:
                # Valid pc4 extracted, but not found in lookup
                record['latitude'] = None
                record['longitude'] = None
                no_match_count += 1
                # Optional: Print warning for specific non-matches
                # if i < 5 or no_match_count < 5: # Limit warnings
                #    print(f"  Warning: No coordinate match found for PC4 '{pc4}' (from '{full_postcode}') in record {i+1}")
        else:
            # Could not extract valid pc4 from record
            record['latitude'] = None
            record['longitude'] = None
            no_postcode_count += 1
            # Optional: Print warning for records missing valid postcode
            # if i < 5 or no_postcode_count < 5: # Limit warnings
            #     print(f"  Warning: Could not extract valid PC4 from postcode '{full_postcode}' in record {i+1}")

    print("\n--- Processing Summary ---")
    print(f"Total records processed: {len(json_data)}")
    print(f"Records successfully matched with coordinates: {match_count}")
    print(f"Records with valid PC4 but no coordinate match: {no_match_count}")
    print(f"Records with missing or invalid postcode format: {no_postcode_count}")

    print(f"\nWriting augmented JSON data to: {json_output_path}...")
    try:
        with open(json_output_path, 'w', encoding='utf-8') as f:
            # Use indent for readability, ensure_ascii=False for special chars
            json.dump(json_data, f, indent=4, ensure_ascii=False)
        print("Successfully wrote output JSON file.")
    except IOError as e:
        print(f"Error: Could not write output JSON file '{json_output_path}': {e}", file=sys.stderr)
    except Exception as e:
        print(f"An unexpected error occurred while writing JSON: {e}", file=sys.stderr)


if __name__ == "__main__":

    try:
        import duckdb
        import pandas
    except ImportError:
        print("Error: duckdb or pandas library is not installed.", file=sys.stderr)
        print("Please install them using: pip install duckdb pandas", file=sys.stderr)
        sys.exit(1)

    print("="*60)
    print(" Geo Data Merger Script ")
    print("="*60)
    
    parquet_geo_file = "georef-netherlands-postcode-pc4.parquet"
    json_input = "zorginstellingen_nijmegen_w_beds_pc.json"
    json_output = "zorginstellingen_nijmegen_w_beds_pc_geo.json"

    # 1. Load Geo Lookup Data
    geo_lookup_data = load_geo_lookup(parquet_geo_file)

    # 2. If lookup loaded successfully, process JSON
    if geo_lookup_data is not None:
        add_geo_to_json(json_input, geo_lookup_data, json_output)
    else:
        print("\nAborted due to error loading geospatial data.", file=sys.stderr)
        sys.exit(1)

    print("\n" + "="*60)
    print("Script finished.")
    print("="*60)

 Geo Data Merger Script 
Loading geospatial data from: georef-netherlands-postcode-pc4.parquet...
Spatial extension loaded.
Executing DuckDB query to extract pc4 and coordinates...
Query finished in 0.00 seconds. Processing 4068 rows.
Created lookup dictionary with 4068 unique 4-digit postcodes.
DuckDB connection closed.

Reading input JSON: zorginstellingen_nijmegen_w_beds_pc.json...
Read 166 records from JSON.
Augmenting JSON records with coordinates...

--- Processing Summary ---
Total records processed: 166
Records successfully matched with coordinates: 166
Records with valid PC4 but no coordinate match: 0
Records with missing or invalid postcode format: 0

Writing augmented JSON data to: zorginstellingen_nijmegen_w_beds_pc_geo.json...
Successfully wrote output JSON file.

Script finished.
